In [4]:
import basedosdados as bd

bd.list_datasets() # deve listar entre os datasets: 'br_anp_precos_combustiveis'

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=yV0TRmbJs2qxVprdMEfXjINh2xEa0t&prompt=consent&access_type=offline

...
dataset_id: 
	br_anp_precos_combustiveis 
...


In [5]:
bd.list_dataset_tables('br_anp_precos_combustiveis') # deve listar 'microdados'


table_id: 
	microdados 
-----------------



In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
BILLING_PROJECT_ID = os.getenv("BILLING_PROJECT_ID")

import basedosdados as bd

QUERY_PROJECT_ID = 'basedosdados'
DATASET_ID = 'br_anp_precos_combustiveis'
TABLE_ID = 'microdados'

def fetch_precos_combustiveis(ano):
    query = f"""
    SELECT *
    FROM `{QUERY_PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE ano = {ano}
    """
    return bd.read_sql(query, billing_project_id=BILLING_PROJECT_ID)

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Crie uma sessão do Spark
spark = ( 
    SparkSession.builder.appName("Cassandra")
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/usr/local/sdkman/candidates/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


In [14]:
# Obtendo a versão do Spark
versao_spark = spark.version
print("Versão do Spark:", versao_spark)

# Outras informações da sessão Spark
print("Configurações Spark:")
for conf, val in spark.sparkContext.getConf().getAll():
    print(conf, ":", val)

Versão do Spark: 3.5.0
Configurações Spark:
spark.jars.packages : com.datastax.spark:spark-cassandra-connector_2.12:3.4.1
spark.app.name : Cassandra
spark.sql.warehouse.dir : file:/workspaces/cassandra-precos-combustiveis/spark-warehouse
spark.executor.id : driver
spark.app.submitTime : 1702312679253
spark.app.initial.jar.urls : spark://ed53c209cc3c:42367/jars/org.hdrhistogram_HdrHistogram-2.1.12.jar,spark://ed53c209cc3c:42367/jars/io.dropwizard.metrics_metrics-core-4.1.18.jar,spark://ed53c209cc3c:42367/jars/org.apache.commons_commons-lang3-3.10.jar,spark://ed53c209cc3c:42367/jars/com.datastax.oss_java-driver-query-builder-4.13.0.jar,spark://ed53c209cc3c:42367/jars/com.datastax.oss_java-driver-mapper-runtime-4.13.0.jar,spark://ed53c209cc3c:42367/jars/org.scala-lang_scala-reflect-2.12.11.jar,spark://ed53c209cc3c:42367/jars/com.github.stephenc.jcip_jcip-annotations-1.0-1.jar,spark://ed53c209cc3c:42367/jars/com.thoughtworks.paranamer_paranamer-2.8.jar,spark://ed53c209cc3c:42367/jars/com.d

In [16]:
import findspark

# Inicializando o Spark no notebook
findspark.init()

# Obtendo o caminho do Spark
caminho_spark = findspark.find()
print("Caminho do Spark:", caminho_spark)

Caminho do Spark: /usr/local/sdkman/candidates/spark/current


In [12]:
from pyspark.sql.functions import when, lit, col

# Propriedades de conexão com o Cassandra
CASSANDRA_CONF = {
    "spark.cassandra.connection.host": "cassandra",
    "spark.cassandra.connection.port": 9042,
}

def ingestao_precos_combustiveis_ano(ano):
    print(':. Carregando o dataframe')
    df = fetch_precos_combustiveis(ano)

    print(':. Convertendo para o dataframe do Spark')
    df_spark = spark.createDataFrame(df)

    print(':. Tratando valores NaN')
    df_spark = (
        df_spark
        .withColumn('preco_compra', when(col('preco_compra') == 'NaN', lit(None)).otherwise(col('preco_compra')))
        .withColumn('preco_venda', when(col('preco_venda') == 'NaN', lit(None)).otherwise(col('preco_venda')))
    ) 

    print(':. Salvando no Cassandra')
    (
        df_spark.write.format("org.apache.spark.sql.cassandra")
        .options(**CASSANDRA_CONF)
        .options(keyspace="anp", table="precos_combustiveis_por_produto_e_uf")
        .mode("append")
        .save()
    )

In [13]:
for ano in range(2004, 2023):
    print(f'Ingestão de dados para o ano {ano}')
    ingestao_precos_combustiveis_ano(ano)


Ingestão de dados para o ano 2004
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2005
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2006
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2007
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2008
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


...

Ingestão de dados para o ano 2018
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2019
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2020
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2021
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


Ingestão de dados para o ano 2022
:. Carregando o dataframe
Downloading: 100%|██████████|
:. Convertendo para o dataframe do Spark
:. Tratando valores NaN
:. Salvando no Cassandra


In [23]:
(
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(**CASSANDRA_CONF)
    .options(keyspace="anp", table="precos_combustiveis_por_produto_e_uf")
    .load()
    .createOrReplaceTempView("vw_precos")
)

spark.sql('select produto, count(*) from vw_precos group by produto').show()

+------------------+--------+
|           produto|count(1)|
+------------------+--------+
|Gasolina Aditivada|  294451|
|            Etanol| 2831126|
|               Gnv|  181106|
|            Diesel| 2156850|
|        Diesel S50|    5901|
|        Diesel S10| 1106498|
|               Glp| 3585616|
|          Gasolina| 3079017|
+------------------+--------+

